### Projekt Python

### Zaimportowanie potrzebnych bibliotek

In [2]:
import pandas as pd
import numpy as np
import requests
import json
import seaborn as sns
import matplotlib.pyplot as plt
import sqlite3

### Pobranie danych piłkarsakich ze strony https://www.football-data.org/


### Wygenerowany klucz API przypisany do zmiennej API_Key

In [4]:
API_Key = '9fae0cdf264049d6b6d0f79be9889c41'

### Stworzenie funkcji do pobrania danych

In [6]:
def get_data_with_api(page):
    url = f"https://api.football-data.org/v4/{page}"
    headers = {"X-Auth-Token": API_Key}
    response = requests.get(url, headers=headers)
    return response.json()

### Pobranie danych "competitions/PL" i użycie funkcji head()

In [8]:
PL_competitions_data = get_data_with_api('competitions/PL')
PL_competitions_json = pd.json_normalize(PL_competitions_data['seasons'])
PL_competitions_json.head(5)

,id,startDate,endDate,currentMatchday,winner,winner.id,winner.name,winner.shortName,winner.tla,winner.crest,winner.address,winner.website,winner.founded,winner.clubColors,winner.venue,winner.lastUpdated
0,2287,2024-08-16,2025-05-25,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1564,2023-08-11,2024-05-19,38.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1490,2022-08-05,2023-05-28,38.0,NaN,65.0,Manchester City FC,Man City,MCI,https://crests.football-data.org/65.png,SportCity Manchester M11 3FF,https://www.mancity.com,1880.0,Sky Blue / White,Etihad Stadium,2022-02-10T19:48:37Z
3,733,2021-08-13,2022-05-22,38.0,NaN,65.0,Manchester City FC,Man City,MCI,https://crests.football-data.org/65.png,SportCity Manchester M11 3FF,https://www.mancity.com,1880.0,Sky Blue / White,Etihad Stadium,2022-02-10T19:48:37Z
4,619,2020-09-12,2021-05-23,38.0,NaN,65.0,Manchester City FC,Man City,MCI,https://crests.football-data.org/65.png,SportCity Manchester M11 3FF,https://www.mancity.com,1880.0,Sky Blue / White,Etihad Stadium,2022-02-10T19:48:37Z


### Wyświetlenie jedynie najważniejszych kolumn, przypisanie ich do Data Frame oraz zamiana nazwy kolumny 'id' na 'comp_id'

In [10]:
PL_competitions_json_filtered = PL_competitions_json[['id', 'startDate', 'endDate', 'winner.id', 'winner.name']]
PL_competitions_df = pd.DataFrame(PL_competitions_json_filtered)
PL_competitions_df.rename(columns={'id': 'comp.id'}, inplace = True)
PL_competitions_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126 entries, 0 to 125
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   comp.id      126 non-null    int64  
 1   startDate    126 non-null    object 
 2   endDate      126 non-null    object 
 3   winner.id    33 non-null     float64
 4   winner.name  33 non-null     object 
dtypes: float64(1), int64(1), object(3)
memory usage: 5.1+ KB


### Usunięcie wierszy z wartościami NaN (wiersze bez winner.id i winner.name jest bezużyteczny)

In [12]:
PL_competitions_df.dropna(inplace = True)
PL_competitions_df.head(10)

,comp.id,startDate,endDate,winner.id,winner.name
2,1490,2022-08-05,2023-05-28,65.0,Manchester City FC
3,733,2021-08-13,2022-05-22,65.0,Manchester City FC
4,619,2020-09-12,2021-05-23,65.0,Manchester City FC
5,468,2019-08-09,2020-07-26,64.0,Liverpool FC
6,151,2018-08-10,2019-05-12,65.0,Manchester City FC
7,23,2017-08-11,2018-05-13,65.0,Manchester City FC
8,256,2016-08-13,2017-05-21,61.0,Chelsea FC
9,257,2015-08-08,2016-05-17,338.0,Leicester City FC
10,258,2014-08-16,2015-05-24,61.0,Chelsea FC
11,259,2013-08-17,2014-05-11,65.0,Manchester City FC


### Pobranie danych Competitions/PL/teams

In [14]:
PL_teams_data = get_data_with_api('competitions/PL/teams')
PL_teams_json = pd.json_normalize(PL_teams_data['teams'])
PL_teams_json.head(3)

,id,name,shortName,tla,crest,address,website,founded,clubColors,venue,...,area.code,area.flag,coach.id,coach.firstName,coach.lastName,coach.name,coach.dateOfBirth,coach.nationality,coach.contract.start,coach.contract.until
0,57,Arsenal FC,Arsenal,ARS,https://crests.football-data.org/57.png,75 Drayton Park London N5 1BU,http://www.arsenal.com,1886,Red / White,Emirates Stadium,...,ENG,https://crests.football-data.org/770.svg,179744,,Mikel Arteta,Mikel Arteta,1982-03-26,Spain,2019-12,2027-06
1,58,Aston Villa FC,Aston Villa,AVL,https://crests.football-data.org/58.png,Villa Park Birmingham B6 6HE,http://www.avfc.co.uk,1872,Claret / Sky Blue,Villa Park,...,ENG,https://crests.football-data.org/770.svg,11616,,Unai Emery,Unai Emery,1971-11-03,Spain,2022-11,2027-06
2,61,Chelsea FC,Chelsea,CHE,https://crests.football-data.org/61.png,Fulham Road London SW6 1HS,http://www.chelseafc.com,1905,Royal Blue / White,Stamford Bridge,...,ENG,https://crests.football-data.org/770.svg,172279,Enzo,Maresca,Enzo Maresca,1980-02-10,Italy,2024-07,2029-06


### Wyświetlenie jedynie najważniejszych kolumn, przypisanie ich do DataFrame oraz posortowanie po roku założenia ('founded')

In [16]:
PL_teams_json_filtered = PL_teams_json[['id','name','founded','shortName','tla']]
PL_teams_df = pd.DataFrame(PL_teams_json_filtered)
PL_teams_df.sort_values('founded', ascending = True)
PL_teams_df.rename(columns={'id': 'team.id'}, inplace = True)
PL_teams_df.reset_index(drop=True, inplace=True)
PL_teams_df.head(5)

,team.id,name,founded,shortName,tla
0,57,Arsenal FC,1886,Arsenal,ARS
1,58,Aston Villa FC,1872,Aston Villa,AVL
2,61,Chelsea FC,1905,Chelsea,CHE
3,62,Everton FC,1878,Everton,EVE
4,63,Fulham FC,1879,Fulham,FUL


### Pobranie danych competitions/PL/matches

In [18]:
PL_competitions_matches_data = get_data_with_api('competitions/PL/matches')
PL_matches_json = pd.json_normalize(PL_competitions_matches_data['matches'])
PL_matches_json.head(3)

,id,utcDate,status,matchday,stage,group,lastUpdated,referees,area.id,area.name,...,awayTeam.shortName,awayTeam.tla,awayTeam.crest,score.winner,score.duration,score.fullTime.home,score.fullTime.away,score.halfTime.home,score.halfTime.away,odds.msg
0,497410,2024-08-16T19:00:00Z,FINISHED,1,REGULAR_SEASON,None,2024-11-03T00:20:44Z,"[{'id': 11446, 'name': 'Robert Jones', 'type':...",2072,England,...,Fulham,FUL,https://crests.football-data.org/63.png,HOME_TEAM,REGULAR,1.0,0.0,0.0,0.0,Activate Odds-Package in User-Panel to retriev...
1,497411,2024-08-17T11:30:00Z,FINISHED,1,REGULAR_SEASON,None,2024-11-03T00:20:44Z,"[{'id': 11396, 'name': 'Tim Robinson', 'type':...",2072,England,...,Liverpool,LIV,https://crests.football-data.org/64.png,AWAY_TEAM,REGULAR,0.0,2.0,0.0,0.0,Activate Odds-Package in User-Panel to retriev...
2,497412,2024-08-17T14:00:00Z,FINISHED,1,REGULAR_SEASON,None,2024-11-03T00:20:44Z,"[{'id': 23568, 'name': 'Jarred Gillett', 'type...",2072,England,...,Wolverhampton,WOL,https://crests.football-data.org/76.png,HOME_TEAM,REGULAR,2.0,0.0,1.0,0.0,Activate Odds-Package in User-Panel to retriev...


### Wyświetlenie jedynie najważniejszych kolumn, przypisanie danych do DataFrame
### oraz zamiana nazwy kolumny 'id' na 'match.id'

In [20]:
PL_matches_json_filtered = PL_matches_json[['id','homeTeam.name', 'awayTeam.name','score.winner',
                            'score.fullTime.home', 'score.fullTime.away']]
PL_matches_df = pd.DataFrame(PL_matches_json_filtered)
PL_matches_df.rename(columns={'id': 'match.id'}, inplace = True)
PL_matches_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 380 entries, 0 to 379
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   match.id             380 non-null    int64  
 1   homeTeam.name        380 non-null    object 
 2   awayTeam.name        380 non-null    object 
 3   score.winner         98 non-null     object 
 4   score.fullTime.home  98 non-null     float64
 5   score.fullTime.away  98 non-null     float64
dtypes: float64(2), int64(1), object(3)
memory usage: 17.9+ KB


### około 300 wierszy ma wartość null, dlatego warto pozbyć się tych wierszy za pomocą funkcji dropna()

In [22]:
PL_matches_df.dropna(inplace = True)
PL_matches_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 98 entries, 0 to 97
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   match.id             98 non-null     int64  
 1   homeTeam.name        98 non-null     object 
 2   awayTeam.name        98 non-null     object 
 3   score.winner         98 non-null     object 
 4   score.fullTime.home  98 non-null     float64
 5   score.fullTime.away  98 non-null     float64
dtypes: float64(2), int64(1), object(3)
memory usage: 5.4+ KB


In [24]:
PL_matches_df.head(10)

,match.id,homeTeam.name,awayTeam.name,score.winner,score.fullTime.home,score.fullTime.away
0,497410,Manchester United FC,Fulham FC,HOME_TEAM,1.0,0.0
1,497411,Ipswich Town FC,Liverpool FC,AWAY_TEAM,0.0,2.0
2,497412,Arsenal FC,Wolverhampton Wanderers FC,HOME_TEAM,2.0,0.0
3,497413,Everton FC,Brighton & Hove Albion FC,AWAY_TEAM,0.0,3.0
4,497414,Newcastle United FC,Southampton FC,HOME_TEAM,1.0,0.0
5,497415,Nottingham Forest FC,AFC Bournemouth,DRAW,1.0,1.0
6,497416,West Ham United FC,Aston Villa FC,AWAY_TEAM,1.0,2.0
7,497417,Brentford FC,Crystal Palace FC,HOME_TEAM,2.0,1.0
8,497418,Chelsea FC,Manchester City FC,AWAY_TEAM,0.0,2.0
9,497419,Leicester City FC,Tottenham Hotspur FC,DRAW,1.0,1.0


### Pobranie danych competitions/PL/standings

In [26]:
PL_competitions_standings_data = get_data_with_api('competitions/PL/standings')
PL_competitions_standings_json = pd.json_normalize(PL_competitions_standings_data['standings'][0]['table'])
PL_competitions_standings_json.head(5)

,position,playedGames,form,won,draw,lost,points,goalsFor,goalsAgainst,goalDifference,team.id,team.name,team.shortName,team.tla,team.crest
0,1,10,None,8,1,1,25,19,6,13,64,Liverpool FC,Liverpool,LIV,https://crests.football-data.org/64.png
1,2,10,None,7,2,1,23,21,11,10,65,Manchester City FC,Man City,MCI,https://crests.football-data.org/65.png
2,3,10,None,6,3,1,21,17,11,6,58,Aston Villa FC,Aston Villa,AVL,https://crests.football-data.org/58.png
3,4,10,None,5,4,1,19,14,7,7,351,Nottingham Forest FC,Nottingham,NOT,https://crests.football-data.org/351.png
4,5,10,None,5,3,2,18,17,11,6,57,Arsenal FC,Arsenal,ARS,https://crests.football-data.org/57.png


### Wyświetlenie jedynie najważniejszych kolumn oraz przypisanie danych do DataFrame

In [28]:
PL_competitions_standings_json_filtered = PL_competitions_standings_json[['position','team.name', 'team.id', 'points', 'won',
                                                                          'draw', 'lost', 'goalsFor', 'goalsAgainst', 'goalDifference']]
PL_competitions_standings_df = pd.DataFrame(PL_competitions_standings_json_filtered)
PL_competitions_standings_df

,position,team.name,team.id,points,won,draw,lost,goalsFor,goalsAgainst,goalDifference
0,1,Liverpool FC,64,25,8,1,1,19,6,13
1,2,Manchester City FC,65,23,7,2,1,21,11,10
2,3,Aston Villa FC,58,21,6,3,1,17,11,6
3,4,Nottingham Forest FC,351,19,5,4,1,14,7,7
4,5,Arsenal FC,57,18,5,3,2,17,11,6
5,6,Chelsea FC,61,17,5,2,2,19,11,8
6,7,Brighton & Hove Albion FC,397,16,4,4,2,17,14,3
7,8,AFC Bournemouth,1044,15,4,3,3,13,12,1
8,9,Newcastle United FC,67,15,4,3,3,10,10,0
9,10,Tottenham Hotspur FC,73,13,4,1,5,18,11,7


### Pobranie danych Competitions/PL/Top Scorers

In [30]:
PL_competitions_TOPScorers = get_data_with_api('competitions/PL/scorers')
PL_competitions_TOPScorers_json = pd.json_normalize(PL_competitions_TOPScorers['scorers'])
PL_competitions_TOPScorers_json.head(3)

,playedMatches,goals,assists,penalties,player.id,player.name,player.firstName,player.lastName,player.dateOfBirth,player.nationality,...,team.name,team.shortName,team.tla,team.crest,team.address,team.website,team.founded,team.clubColors,team.venue,team.lastUpdated
0,10,11,NaN,1.0,38101,Erling Haaland,Erling,Haaland,2000-07-21,Norway,...,Manchester City FC,Man City,MCI,https://crests.football-data.org/65.png,SportCity Manchester M11 3FF,https://www.mancity.com,1880,Sky Blue / White,Etihad Stadium,2022-02-10T19:48:37Z
1,10,8,NaN,1.0,8057,Chris Wood,Chris,Wood,1991-12-07,New Zealand,...,Nottingham Forest FC,Nottingham,NOT,https://crests.football-data.org/351.png,"Pavilion Road, West Bridgeford Nottingham NG2 5GJ",http://www.nottinghamforest.co.uk,1865,Red / White,The City Ground,2022-07-26T17:53:21Z
2,9,8,NaN,2.0,8626,Bryan Mbeumo,Bryan,Mbeumo,1999-08-07,Cameroon,...,Brentford FC,Brentford,BRE,https://crests.football-data.org/402.png,Braemar Road Brentford TW8 0NT,http://www.brentfordfc.co.uk,1889,Red / White / Black,Griffin Park,2022-04-03T16:24:00Z


### Wyświetlenie naajważniejszych kolumn i przypisanie ich do DataFrame

In [32]:
PL_competitions_TOPScorers_json_filtered = PL_competitions_TOPScorers_json[['player.name', 'player.dateOfBirth', 'team.name', 'team.tla',
                                                                            'goals', 'playedMatches']]
PL_competitions_TOPScorers_df = pd.DataFrame(PL_competitions_TOPScorers_json_filtered)

PL_competitions_TOPScorers_df

,player.name,player.dateOfBirth,team.name,team.tla,goals,playedMatches
0,Erling Haaland,2000-07-21,Manchester City FC,MCI,11,10
1,Chris Wood,1991-12-07,Nottingham Forest FC,NOT,8,10
2,Bryan Mbeumo,1999-08-07,Brentford FC,BRE,8,9
3,Mohamed Salah,1992-06-15,Liverpool FC,LIV,7,10
4,Cole Palmer,2002-05-06,Chelsea FC,CHE,7,9
5,Danny Welbeck,1990-11-26,Brighton & Hove Albion FC,BHA,6,10
6,Nicolas Jackson,2001-06-20,Chelsea FC,CHE,6,9
7,Luis Díaz,1997-01-13,Liverpool FC,LIV,5,10
8,Liam Delap,2003-02-08,Ipswich Town FC,IPS,5,10
9,Ollie Watkins,1995-12-30,Aston Villa FC,AVL,5,10


### Wczytanie wszystkich DataFrames do Sqlite3

In [34]:
import sqlite3

conn = sqlite3.connect('PL_database.db')

dataframes = {
    'PL_competitions': PL_competitions_df,
    'PL_teams': PL_teams_df,
    'PL_standings': PL_competitions_standings_df,
    'PL_TOPscorers': PL_competitions_TOPScorers_df,
    'PL_matches': PL_matches_df
}

for table_name, df in dataframes.items():
    df.to_sql(table_name, conn, if_exists='replace', index=False)

conn.close()